In [1]:
import pandas as pd
import numpy as np
from utils.connect import get_data
from utils.tools import  CustomOneHotEncoder, CustomRobustScaler,CustomLabelEncoder,train_test_val_split
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow_recommenders as tfrs
import tensorflow as tf
from typing import Dict, Text

ProgrammingError: ('42000', '[42000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Cannot open database "Itunasam" requested by the login. The login failed. (4060) (SQLDriverConnect)')

In [2]:
sql = """
select Trabajador,Hora,  COUNT(*) as Rating from vhorario group by Trabajador, Hora
order by Rating desc;

"""
df_all=get_data(sql)
df_all

/home/carlos/projects/tunasam/notebook/utils/connect.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql, cnxn)


,Trabajador,Hora,Rating
0,HRS285,MA-9,25
1,DOC184,MI-10,24
2,ASE526,MA-17,21
3,ASE526,MA-16,20
4,JRO001,MI-9,19
...,...,...,...
38529,DMDB02,MI-19,1
38530,VCAI00,JU-21,1
38531,PODA01,MI-16,1
38532,ASJE02,LU-18,1


In [3]:
ratings = tf.data.Dataset.from_tensor_slices(dict(df_all))
movies = tf.data.Dataset.from_tensor_slices(df_all["Hora"].unique())

ratings = ratings.map(
    lambda x: {
        "original_title": x["Hora"],
        "userId": x["Trabajador"],
        "rating": float(x["Rating"]),
    }
)



2023-08-31 10:40:02.096145: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [4]:
print('Total Data: {}'.format(len(ratings)))

tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = ratings.take(30_000)
test = ratings.skip(30_000).take(8_382)

Total Data: 38534


In [5]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000).map(lambda x: x["userId"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

print('Unique Movies: {}'.format(len(unique_movie_titles)))
print('Unique users: {}'.format(len(unique_user_ids)))


2023-08-31 10:40:02.395842: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [90]
	 [[{{node Placeholder/_0}}]]
2023-08-31 10:40:02.447706: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [38534]
	 [[{{node Placeholder/_2}}]]


Unique Movies: 90
Unique users: 906


In [6]:
class MovieModel(tfrs.models.Model):
    def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
        # We take the loss weights in the constructor: this allows us to instantiate
        # several model objects with different loss weights.

        super().__init__()

        embedding_dimension = 64

        # User and movie models.
        self.movie_model: tf.keras.layers.Layer = tf.keras.Sequential(
            [
                tf.keras.layers.StringLookup(
                    vocabulary=unique_movie_titles, mask_token=None
                ),
                tf.keras.layers.Embedding(
                    len(unique_movie_titles) + 1, embedding_dimension
                ),
            ]
        )
        self.user_model: tf.keras.layers.Layer = tf.keras.Sequential(
            [
                tf.keras.layers.StringLookup(
                    vocabulary=unique_user_ids, mask_token=None
                ),
                tf.keras.layers.Embedding(
                    len(unique_user_ids) + 1, embedding_dimension
                ),
            ]
        )

        # A small model to take in user and movie embeddings and predict ratings.
        # We can make this as complicated as we want as long as we output a scalar
        # as our prediction.
        self.rating_model = tf.keras.Sequential(
            [
                tf.keras.layers.Dense(256, activation="relu"),
                tf.keras.layers.Dense(128, activation="relu"),
                tf.keras.layers.Dense(1),
            ]
        )

        # The tasks.
        self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()],
        )
        self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=movies.batch(128).map(self.movie_model)
            )
        )

        # The loss weights.
        self.rating_weight = rating_weight
        self.retrieval_weight = retrieval_weight

    def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
        # We pick out the user features and pass them into the user model.
        user_embeddings = self.user_model(features["userId"])
        # And pick out the movie features and pass them into the movie model.
        movie_embeddings = self.movie_model(features["original_title"])

        return (
            user_embeddings,
            movie_embeddings,
            # We apply the multi-layered rating model to a concatentation of
            # user and movie embeddings.
            self.rating_model(tf.concat([user_embeddings, movie_embeddings], axis=1)),
        )

    def compute_loss(
        self, features: Dict[Text, tf.Tensor], training=False
    ) -> tf.Tensor:
        ratings = features.pop("rating")

        user_embeddings, movie_embeddings, rating_predictions = self(features)

        # We compute the loss for each task.
        rating_loss = self.rating_task(
            labels=ratings,
            predictions=rating_predictions,
        )
        retrieval_loss = self.retrieval_task(user_embeddings, movie_embeddings)

        # And combine them using the loss weights.
        return self.rating_weight * rating_loss + self.retrieval_weight * retrieval_loss

In [7]:
model = MovieModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

cached_train = train.shuffle(100_000).batch(1_000).cache()
cached_test = test.batch(1_000).cache()

model.fit(cached_train, epochs=50)

Epoch 1/50


2023-08-31 10:40:03.769592: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [38534]
	 [[{{node Placeholder/_2}}]]


 1/30 [>.............................] - ETA: 2:12 - root_mean_squared_error: 3.8273 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0070 - factorized_top_k/top_10_categorical_accuracy: 0.0460 - factorized_top_k/top_50_categorical_accuracy: 0.5640 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 6922.4858 - regularization_loss: 0.0000e+00 - total_loss: 6922.4858

2023-08-31 10:40:08.189964: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [1000,64]
	 [[{{node Placeholder/_11}}]]


 2/30 [=>............................] - ETA: 6s - root_mean_squared_error: 3.3903 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0165 - factorized_top_k/top_10_categorical_accuracy: 0.0565 - factorized_top_k/top_50_categorical_accuracy: 0.5545 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 6919.5828 - regularization_loss: 0.0000e+00 - total_loss: 6919.5828  

...

2023-08-31 10:46:41.617079: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype resource
	 [[{{node Placeholder/_7}}]]
2023-08-31 10:46:41.774358: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype resource
	 [[{{node Placeholder/_7}}]]


28/30 [===========================>..] - ETA: 0s - root_mean_squared_error: 1.0168 - factorized_top_k/top_1_categorical_accuracy: 6.7857e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0918 - factorized_top_k/top_10_categorical_accuracy: 0.2077 - factorized_top_k/top_50_categorical_accuracy: 0.9132 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 6412.5160 - regularization_loss: 0.0000e+00 - total_loss: 6412.5160

2023-08-31 10:46:42.042557: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype resource
	 [[{{node Placeholder/_7}}]]
2023-08-31 10:46:42.228386: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype resource
	 [[{{node Placeholder/_7}}]]


30/30 [==============================] - 5s 171ms/step - root_mean_squared_error: 1.0110 - factorized_top_k/top_1_categorical_accuracy: 7.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0919 - factorized_top_k/top_10_categorical_accuracy: 0.2085 - factorized_top_k/top_50_categorical_accuracy: 0.9126 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 6413.3936 - regularization_loss: 0.0000e+00 - total_loss: 6413.3936


2023-08-31 10:46:42.504024: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype resource
	 [[{{node Placeholder/_7}}]]


In [8]:
metrics = model.evaluate(cached_test, return_dict=True)

print(
    f"\nRetrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}"
)
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}")

2023-08-31 10:46:42.638669: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [38534]
	 [[{{node Placeholder/_1}}]]


2/9 [=====>........................] - ETA: 0s - root_mean_squared_error: 0.9691 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0045 - factorized_top_k/top_10_categorical_accuracy: 0.0365 - factorized_top_k/top_50_categorical_accuracy: 0.4955 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 8811.5645 - regularization_loss: 0.0000e+00 - total_loss: 8811.5645 

2023-08-31 10:46:45.267379: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_9' with dtype resource
	 [[{{node Placeholder/_9}}]]
2023-08-31 10:46:45.355272: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [1000,64]
	 [[{{node Placeholder/_11}}]]
2023-08-31 10:46:45.458816: W tensorflow/core/data/root_dataset.cc:273] Optimization loop failed: CANCELLED: Operation was cancelled


3/9 [=========>....................] - ETA: 0s - root_mean_squared_error: 0.9597 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0037 - factorized_top_k/top_10_categorical_accuracy: 0.0360 - factorized_top_k/top_50_categorical_accuracy: 0.4987 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 8829.3577 - regularization_loss: 0.0000e+00 - total_loss: 8829.3577

2023-08-31 10:46:45.507469: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype resource
	 [[{{node Placeholder/_5}}]]
2023-08-31 10:46:45.666309: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype resource
	 [[{{node Placeholder/_5}}]]


...

7/9 [======================>.......] - ETA: 0s - root_mean_squared_error: 0.9574 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0044 - factorized_top_k/top_10_categorical_accuracy: 0.0390 - factorized_top_k/top_50_categorical_accuracy: 0.4913 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 8844.0541 - regularization_loss: 0.0000e+00 - total_loss: 8844.0541

2023-08-31 10:46:46.110335: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype resource
	 [[{{node Placeholder/_5}}]]
2023-08-31 10:46:46.302657: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [1000,64]
	 [[{{node Placeholder/_11}}]]


9/9 [==============================] - 4s 156ms/step - root_mean_squared_error: 0.9562 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0047 - factorized_top_k/top_10_categorical_accuracy: 0.0385 - factorized_top_k/top_50_categorical_accuracy: 0.4877 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 7688.7346 - regularization_loss: 0.0000e+00 - total_loss: 7688.7346


2023-08-31 10:46:46.471784: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [382,64]
	 [[{{node Placeholder/_11}}]]



Retrieval top-100 accuracy: 1.000
Ranking RMSE: 0.956


In [9]:
def predict_movie(user, top_n=3):
    # Create a model that takes in raw query features, and
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
    # recommends movies out of the entire movies dataset.
    index.index_from_dataset(
      tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
    )

    # Get recommendations.
    _, titles = index(tf.constant([str(user)]))
    
    print('Top {} recommendations for user {}:\n'.format(top_n, user))
    for i, title in enumerate(titles[0, :top_n].numpy()):
        print('{}. {}'.format(i+1, title.decode("utf-8")))

def predict_rating(user, movie):
    trained_movie_embeddings, trained_user_embeddings, predicted_rating = model({
          "userId": np.array([str(user)]),
          "original_title": np.array([movie])
      })
    print("Predicted rating for {}: {}".format(movie, predicted_rating.numpy()[0][0]))

def best_predictions(user):
    # Create a model that takes in raw query features, and
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
    # recommends movies out of the entire movies dataset.
    index.index_from_dataset(
      tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
    )

    # Get recommendations.
    _, titles = index(tf.constant([str(user)]))
    return titles[0, :3].numpy()
    

In [10]:
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)
_, titles = index(tf.constant([str("HRS285")]))

titles.numpy()

2023-08-31 10:46:49.106859: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype resource
	 [[{{node Placeholder/_10}}]]
2023-08-31 10:46:49.107391: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype string and shape [90]
	 [[{{node Placeholder/_5}}]]


array([[b'MI-14', b'MA-17', b'JU-12', b'MI-7', b'MA-10', b'MI-15',
        b'JU-17', b'JU-11', b'MA-9', b'JU-16']], dtype=object)

In [11]:
predict_movie('HRS285', 5)
predict_rating('HRS285','MA-9')


Top 5 recommendations for user HRS285:

1. MI-14
2. MA-17
3. JU-12
4. MI-7
5. MA-10
Predicted rating for MA-9: 23.787525177001953


2023-08-31 10:46:49.215220: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [90]
	 [[{{node Placeholder/_0}}]]
2023-08-31 10:46:49.215581: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype string and shape [90]
	 [[{{node Placeholder/_5}}]]
